# Instructions
The following code was designed in order to calculate the number of cells within an image or set of images.

### A few pertinent features of the code:
This code calculates the number of counted cells on up to two channels, as well as overlapping cells across channels.
In addition to cell count, the locations of counted cells can be saved.
Lastly, a region of interest based upon a user-made mask can be specified and cell counting can be restricted to this region of interest.  The size of the region of interest in pixels is returned. 

### Folder Structure

For this code to work files should be organized in a specific manner.
Directory_Main (the path of which is to be defined by the user... actual directory name can be whatever you want) should contain the subfolders "Ch1","Ch2",and "ROI" (these names matter).
If you are not using ROIs, this folder can be excluded.
Similarly, if only one channel is being examined, Ch1/Ch2 can be exluded.
Ch1 and Ch2 folders should contain single-channel .tif images with files of the same name, though some suffix identifying the channel (e.g. "_Ch1") is fine.  This is required in order for images for each channel of a multichannel image to be matched.
ROI should contain .tif images with ROI masks, described below.  These should be named similarly to files in Ch1 and Ch2.
Example filenames under Ch1, Ch2, and ROI: "Mouse1_Image1_Ch1.tif","Mouse1_Image1_Ch2.tif",""Mouse1_Image1_ROI.tif"

### Making ROIs
Regions of interest can be drawn using ROIdrawer.ipynb.  Alternatively, one can make similar binary masks using ImageJ. In ImageJ:
#1 - Open the image in ImageJ
#2 - Use your preferred selection tool (I like freehand selection tool) to outline the region of interest
#3 - Go to edit > selection > create mask
#4 - Save the mask as .tif file

### Requirements
See README for dependencies and conda environment install instructions

### What type of cells can this count?
Any semi-circular cell that is more or less filled by the fluorescent label will be cable of being counted.  If the cells are only visible by their perimeter than this code as is may not work. In this case, a process should be applied to either a) fill the internal component of the cell after thresholding or b) use smoothing or morphological opening to increase the internal of the cell. Although watershed can separate cells to some extent, a large degree ov overlap is unable to be overcome.

## Setting parameters:
All parameters required to be set by the user is set in the first cell of code.  
1 - Cell Diameter:  This is the average cell diameter in pixel units.  Can be obtained easily with ImageJ measurement tool.
2 - Threshold:  This is to be obtained using the optimization procedure or by setting by eye (not advised).
3 - Paricle Minimum: After thresholding this serves to erase excessively small points.  Proportion of average cell size permitted.  Average cell size is assumed to be square of diamater for rough approximation
4 - Use ROI:  Are you using an ROI or analyzing the entire image?
5- Use Watershed: Watershed procedure attempts to separate adjoining cells. 


# 1. Load Necessary Packages
The following code loads necessary packages and need not be changed by the user

In [ ]:
%load_ext autoreload
%autoreload 2
import pylab 
import os
import fnmatch
import cv2
import numpy as np
import mahotas as mh 
import pandas as pd
import holoviews as hv
import CellCounter_Functions as cc

# 2. User Specifies Parameters and Options for Running
Below the user specifies various parameters relevant to analysis.  You can safely ignore settings parameters that do not pertain to what you are doing; however, they do require some value.

***Windows Users:*** Place an 'r' in front directory path (e.g. r"zp\Videos") to avoid mishandling of forward slashes.

In [ ]:
#Define directory of working environment (Directory.Main)
dirinfo = {
    'main' : "/Users/zachpennington/Desktop/images/"
}

#Set Parameters
params = {
    'ch1_diam' : 19 ,#Average Cell Diameter in pixel units.  Must be integer value.  
    'ch1_thresh' : 510 ,#Set threshold to integer value based upon CellCounter_Optimization.ipynb
    'ch2_diam' : 10,#Average Cell Diameter in pixel units.  Must be integer value.  
    'ch2_thresh' : 1060 ,#Set threshold to integer value based upon CellCounter_Optimization.ipynb
    'particle_min' : 0.2 ,#After thresholding this serves to erase excessively small points.  Proportion of average cell size permitted.  Average cell size is assumed to be square of diamater for rough approximation.  0.2 has worked well for me
    'overlap' : .5 #When measuring overlap of two channels, this is the minimum amount of overlap permitted, as a proportion of average cell size for channel with smaller diameter cells.  Average cell size is assumed to be square of diamater for rough approximation.
}

dirinfo = cc.getdirinfo(dirinfo)
dirinfo

# 3. Display Example Process for One Channel (Optional)
The code helps display the processing results for a single image.  The original image, the background subtracted and smoothed image, the thresholded image, and the final cell locations are displayed.   By default displays results from first image of first channel.  Primarily used for troubleshooting.  Can't view ROI this way

Set channel equal to "Ch1" or "Ch2".  Additionally, file number can be set based upon alphabetical ordering of files (0 is first file, etc)

To toggle whether an ROI or Watershed are to be used, set to True or False

In [ ]:
%%output size=120

#Specify Image to Look at Below
Channel = "Ch1" #Specify Ch1 or Ch2
file = 0 #Specify index in folder.  0 by default

#call function to count cells
count_out = cc.Count(
    file,Channel,params,dirinfo,
    UseROI=True,
    UseWatershed=True
)
display = cc.display_count(count_out)
display.cols(2)

# 4. Display Example Process of Overlap (Optional) 
The code helps display the overlap results for a single set of images (where each image is a different channel).  The original images, the final cell locations, and the overlapping cells are displayed.

In [ ]:
%%output size=100
#Specify Image to Look at Below
file = 0 #Specify index in folder.  0 by default


#call function to count cells
count_out1 = cc.Count(
    file,"Ch1",params,dirinfo,
    UseROI=True,
    UseWatershed=True
)
count_out2 = cc.Count(
    file,"Ch2",params,dirinfo,
    UseROI=True,
    UseWatershed=True
)
merge_out = cc.Merge(count_out1['cells'],count_out2['cells'],params)
display = cc.display_merge(count_out1,count_out2,merge_out)
display.cols(2)

# 5. Count Channel 1
Count all cells for channel 1 images and save to disk

In [ ]:
Ch1_Counts = cc.Count_folder(
    dirinfo,
    params,
    Channel="Ch1",
    UseROI=True,
    UseWatershed=True,
    SaveIntensities=True
)
Ch1_Counts.to_csv(os.path.join(os.path.normpath(dirinfo['output']), "Ch1_Counts.csv"))
Ch1_Counts

# 6. Count Channel 2
Count all cells for channel 2 images and save to disk

In [ ]:
Ch2_Counts = cc.Count_folder(
    dirinfo,
    params,
    Channel="Ch2",
    UseROI=True,
    UseWatershed=True,
    SaveIntensities=True
)
Ch2_Counts.to_csv(os.path.join(os.path.normpath(dirinfo['output']), "Ch2_Counts.csv"))
Ch2_Counts

# 7. Count Overlapping Cells from Channels 1 and 2
Count cells overlapping in Channel 1 and Channel 2 and save results to disk
**Note:** Channels 1 and 2 must be counted before doing this
Count overlapping cells for channel 1 and channel 2

In [ ]:
Merge_Counts = cc.Merge_folder(dirinfo,params)
Merge_Counts.to_csv(os.path.join(os.path.normpath(dirinfo['output']), "Merge_Counts.csv"))
Merge_Counts

# 8. Create files with all cell info

The following will create csvs containing information for each counted cell (size and intensity) per channel and be saved to Ch1_CellInfo.csv and Ch2_CellInfo.csv.  Additionally, it will create a file (Merge_CellInfo.csv), defining the mapping between channels.

In [ ]:
cc.extract_cellinfo(dirinfo)

In [ ]:
idx = np.asarray([not ''.join([x.split('c1_ORG.tif')[0], 'c3_ORG_mask_CEA.tif']) in dirinfo['roi_fnames'] for x in dirinfo['ch1_fnames']])
np.asarray(dirinfo['ch1_fnames'])[idx==True]